# Clean Data Dashboard Analytics - Laurent Project
## Advanced Analytics Dashboard for Inventory and Revenue Management

This notebook provides comprehensive analytics including:
- Strategic Overview with KPIs
- Lost Revenue Analysis
- Stock Management
- Pricing Analysis
- Actionable Recommendations
- Advanced Analytics (ABC Analysis, Segmentation)
- Brand Insights
- Data Explorer

## 1. Import Libraries and Configuration

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import numpy as np
from io import BytesIO
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

st.set_page_config(page_title="Advanced Analytics Dashboard", page_icon="📊", layout="wide")

## 2. Custom CSS Styling

In [ ]:
st.markdown("""
    <style>
    ::-webkit-scrollbar {width: 12px; height: 12px;}
    ::-webkit-scrollbar-track {background: linear-gradient(to bottom, #f1f1f1, #e0e0e0); border-radius: 10px;}
    ::-webkit-scrollbar-thumb {background: linear-gradient(to bottom, #888, #555); border-radius: 10px;}
    ::-webkit-scrollbar-thumb:hover {background: linear-gradient(to bottom, #555, #333);}
    .alert-critical {background-color: #ff4444; color: white; padding: 15px; border-radius: 10px; margin: 10px 0; box-shadow: 0 4px 6px rgba(0,0,0,0.1);}
    .alert-warning {background-color: #ffaa00; color: white; padding: 15px; border-radius: 10px; margin: 10px 0; box-shadow: 0 4px 6px rgba(0,0,0,0.1);}
    .alert-success {background-color: #00C851; color: white; padding: 15px; border-radius: 10px; margin: 10px 0; box-shadow: 0 4px 6px rgba(0,0,0,0.1);}
    .metric-card {background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; color: white; box-shadow: 0 4px 6px rgba(0,0,0,0.1);}
    .big-number {font-size: 48px; font-weight: bold; margin: 10px 0;}
    .insight-box {background: #f8f9fa; padding: 20px; border-left: 5px solid #667eea; border-radius: 5px; margin: 15px 0;}
    </style>
""", unsafe_allow_html=True)

## 3. Data Loading Function

In [ ]:
@st.cache_data
def load_data(file_path):
    try:
        df = pd.read_csv(file_path)

        # Remove unwanted columns
        columns_to_drop = ['product_id', 'offer_offer_id']
        df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

        # Recreate product_id as unique identifier (required for dashboard)
        if 'product_id' not in df.columns:
            df['product_id'] = range(len(df))

        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
        if 'offer_price' in df.columns:
            df['offer_price'] = pd.to_numeric(df['offer_price'], errors='coerce')
        if 'stock' in df.columns:
            df['stock'] = pd.to_numeric(df['stock'], errors='coerce')
        if 'offer_in_stock' in df.columns:
            # Convert to boolean (True/False) to avoid type errors
            df['offer_in_stock'] = df['offer_in_stock'].astype(bool)

        # Normalize brand names (merge duplicates)
        if 'brand' in df.columns:
            # Clean whitespace
            df['brand'] = df['brand'].str.strip()
            # Replace all variants
            df['brand'] = df['brand'].str.replace('AUXPORTESDUNATUREL', 'AUX PORTES DU NATUREL', regex=False)
            df['brand'] = df['brand'].replace('AUXPORTESDUNATUREL', 'AUX PORTES DU NATUREL')

        return df
    except Exception as e:
        st.error(f"Error: {e}")
        return None

## 4. Revenue Metrics Functions

In [ ]:
def calculate_revenue_metrics(df):
    metrics = {}
    if 'offer_price' in df.columns and 'stock' in df.columns:
        df_temp = df.dropna(subset=['offer_price', 'stock'])
        metrics['total_potential_revenue'] = (df_temp['offer_price'] * df_temp['stock']).sum()
        
        if 'offer_in_stock' in df.columns:
            out_of_stock = df[df['offer_in_stock'] == False]
            avg_stock = df_temp['stock'].mean() if len(df_temp) > 0 else 5
            metrics['lost_revenue'] = (out_of_stock['offer_price'] * avg_stock).sum()
            total_possible = metrics['total_potential_revenue'] + metrics['lost_revenue']
            metrics['lost_revenue_pct'] = (metrics['lost_revenue'] / total_possible * 100) if total_possible > 0 else 0
    
    if 'stock' in df.columns:
        low_stock = df[df['stock'] < 10]
        metrics['revenue_at_risk'] = (low_stock['offer_price'] * low_stock['stock']).sum()
        metrics['products_at_risk'] = len(low_stock)
        
        # New metrics
        overstock = df[df['stock'] > 100]
        metrics['overstock_value'] = (overstock['offer_price'] * overstock['stock']).sum()
        metrics['overstock_count'] = len(overstock)
        
        metrics['optimal_stock_level'] = df_temp['stock'].quantile(0.75) if len(df_temp) > 0 else 20
        metrics['stock_turnover'] = metrics['total_potential_revenue'] / (df_temp['stock'].sum() + 1)
    
    return metrics

def find_revenue_opportunities(df):
    opportunities = []
    if 'offer_in_stock' in df.columns and 'offer_price' in df.columns:
        out_of_stock = df[df['offer_in_stock'] == False].copy()
        if len(out_of_stock) > 0:
            out_of_stock['potential_revenue'] = out_of_stock['offer_price'] * 10
            opportunities.append(('out_of_stock', out_of_stock.nlargest(20, 'potential_revenue')))
        
        if 'stock' in df.columns:
            low_stock = df[(df['stock'] < 10) & (df['stock'] > 0)].copy()
            if len(low_stock) > 0:
                low_stock['additional_revenue'] = low_stock['offer_price'] * (20 - low_stock['stock'])
                opportunities.append(('low_stock', low_stock.nlargest(20, 'additional_revenue')))
        
        in_stock = df[df['offer_in_stock'] == True].copy()
        if len(in_stock) > 0 and 'stock' in df.columns:
            in_stock['current_revenue'] = in_stock['offer_price'] * in_stock['stock']
            opportunities.append(('top_performers', in_stock.nlargest(20, 'current_revenue')))
    
    return opportunities

## 5. Analysis Functions

In [ ]:
def analyse_abc(df):
    if 'offer_price' not in df.columns or 'stock' not in df.columns:
        return None
    
    df_abc = df.copy()
    df_abc['revenue'] = df_abc['offer_price'] * df_abc['stock']
    df_abc = df_abc.dropna(subset=['revenue'])
    df_abc = df_abc.sort_values('revenue', ascending=False)
    df_abc['cumulative_revenue'] = df_abc['revenue'].cumsum()
    total_revenue = df_abc['revenue'].sum()
    df_abc['cumulative_pct'] = (df_abc['cumulative_revenue'] / total_revenue * 100)
    df_abc['ABC_Category'] = 'C'
    df_abc.loc[df_abc['cumulative_pct'] <= 80, 'ABC_Category'] = 'A'
    df_abc.loc[(df_abc['cumulative_pct'] > 80) & (df_abc['cumulative_pct'] <= 95), 'ABC_Category'] = 'B'
    
    return df_abc

def predict_stockouts(df, days_ahead=30):
    if 'stock' not in df.columns or 'date' not in df.columns:
        return None
    
    predictions = []
    for product_id in df['product_id'].unique():
        product_data = df[df['product_id'] == product_id].sort_values('date')
        
        if len(product_data) < 3:
            continue
        
        stock_values = product_data['stock'].values
        if len(stock_values) < 2:
            continue
        
        x = np.arange(len(stock_values))
        if len(x) > 1:
            slope = np.polyfit(x, stock_values, 1)[0]
            current_stock = stock_values[-1]
            
            if slope < 0 and current_stock > 0:
                days_to_stockout = int(-current_stock / slope)
                
                if 0 < days_to_stockout <= days_ahead:
                    predictions.append({
                        'product_id': product_id,
                        'current_stock': current_stock,
                        'daily_decrease': -slope,
                        'days_to_stockout': days_to_stockout,
                        'risk_level': 'High' if days_to_stockout <= 7 else 'Medium' if days_to_stockout <= 14 else 'Low'
                    })
    
    return pd.DataFrame(predictions) if predictions else None

def analyze_price_stock_correlation(df):
    if 'offer_price' in df.columns and 'stock' in df.columns:
        corr = df[['offer_price', 'stock']].corr().iloc[0, 1]
        return corr
    return None

def get_top_revenue_products(df, n=20):
    if all(c in df.columns for c in ['offer_price', 'stock', 'title', 'brand']):
        df_rev = df.copy()
        df_rev['revenue'] = df_rev['offer_price'] * df_rev['stock']
        return df_rev.nlargest(n, 'revenue')[['title', 'brand', 'offer_price', 'stock', 'revenue']]
    return None

def calculate_inventory_health(df):
    health_score = 100
    
    if 'offer_in_stock' in df.columns:
        out_of_stock_pct = (df['offer_in_stock'] == False).sum() / len(df) * 100
        health_score -= out_of_stock_pct * 0.5
    
    if 'stock' in df.columns:
        low_stock_pct = (df['stock'] < 10).sum() / len(df) * 100
        health_score -= low_stock_pct * 0.3
        
        overstock_pct = (df['stock'] > 100).sum() / len(df) * 100
        health_score -= overstock_pct * 0.2
    
    return max(0, min(100, health_score))

## 6. Navigation and Data Loading

In [ ]:
st.sidebar.title("🎯 Navigation")
page = st.sidebar.radio("Pages:", [
    "🏠 Strategic Overview",
    "💰 Lost Revenue & Opportunities",
    "📦 Stock Management",
    "💵 Pricing Analysis",
    "🎯 Actionable Recommendations",
    "⚡ Advanced Analytics",
    "🔍 Brand Insights",
    "📋 Data Explorer"
])

df = load_data('all_data_laurent2.csv')

## 7. Filters Setup

In [ ]:
if df is not None:
    st.sidebar.markdown("---")
    st.sidebar.header("🔍 Filters")

    df_filtered = df.copy()

    if 'brand' in df.columns:
        brands = ['All'] + sorted(df['brand'].dropna().unique().tolist())
        selected_brand = st.sidebar.selectbox("Brand", brands)
        if selected_brand != 'All':
            df_filtered = df_filtered[df_filtered['brand'] == selected_brand]

    if 'offer_price' in df.columns:
        min_p, max_p = float(df['offer_price'].min()), float(df['offer_price'].max())
        price_range = st.sidebar.slider("Price ($)", min_p, max_p, (min_p, max_p))
        df_filtered = df_filtered[(df_filtered['offer_price'] >= price_range[0]) &
                                  (df_filtered['offer_price'] <= price_range[1])]

    if 'offer_in_stock' in df.columns:
        stock_opts = st.sidebar.multiselect("Status", ['In Stock', 'Out of Stock'], ['In Stock', 'Out of Stock'])
        if 'In Stock' in stock_opts and 'Out of Stock' not in stock_opts:
            df_filtered = df_filtered[df_filtered['offer_in_stock'] == True]
        elif 'Out of Stock' in stock_opts and 'In Stock' not in stock_opts:
            df_filtered = df_filtered[df_filtered['offer_in_stock'] == False]
    
    revenue_metrics = calculate_revenue_metrics(df_filtered)
    opportunities = find_revenue_opportunities(df_filtered)
    inventory_health = calculate_inventory_health(df_filtered)
else:
    st.error("❌ Unable to load file 'all_data_laurent2.csv'")
    st.info("Make sure the file is in the same directory as this script.")

## 8. Page: Strategic Overview

Display main KPIs, critical alerts, and visual analysis of the inventory and revenue.

In [ ]:
# Note: Due to the interactive nature of Streamlit, the full page implementations
# require the Streamlit runtime to execute properly.
# This notebook structure shows how the code is organized.
# For full execution, run: streamlit run dashboard_laurent.py

# The complete implementation includes all 8 pages with their visualizations and analytics.
# Each page is conditionally rendered based on the selected navigation option.

## 9. Footer

In [ ]:
st.sidebar.markdown("---")
st.sidebar.markdown(f"📅 Last updated: {datetime.now().strftime('%d/%m/%Y %H:%M')}")
st.sidebar.markdown("🤖 Analytics Dashboard v2.0")
st.sidebar.info("💡 Use filters to refine your analyses")

---

## Usage Instructions

### Running as Streamlit App:
```bash
streamlit run dashboard_laurent.py
```

### Running in Jupyter:
This notebook contains the code structure. For full interactive experience with Streamlit features (navigation, filters, dynamic charts), use the Python script with Streamlit.

### Data Requirements:
- File: `all_data_laurent2.csv`
- Required columns: `offer_price`, `stock`, `offer_in_stock`, `brand`, `title`, `date`

### Features:
1. **Strategic Overview**: Main KPIs and executive dashboard
2. **Lost Revenue Analysis**: Identify revenue opportunities from stockouts
3. **Stock Management**: Optimize inventory levels
4. **Pricing Analysis**: Strategic pricing insights
5. **Recommendations**: AI-powered actionable insights
6. **Advanced Analytics**: ABC analysis, segmentation, exports
7. **Brand Insights**: Deep-dive into brand performance
8. **Data Explorer**: Custom data views and exports